In [145]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np
import math
import json
from spacy.en import English
import urllib.parse
nlp = spacy.load('en')

In [328]:
#Create SQLite
conn = sqlite3.connect('songlyrics.db')
df = pd.read_sql_query("select * from songlyrics_url where status=1;",conn)


In [33]:
soup = BeautifulSoup(df['fullhtml'][3],"lxml")
df['fullhtml'][10]
lyrics = soup.find('p',attrs={'id':'songLyricsDiv'})
lyric = lyrics.get_text()

In [93]:
lyric_clean = re.sub(r'\n[\(]?Chorus[:]?[\)]?[ ]?\n',"",lyric)
lyric_final = lyric_clean.replace("\n\n","\n").replace("\n",".\n").replace(" .",".")

In [94]:
doc = nlp(lyric_final)

In [91]:
for s in doc.sents:
    print(s)
    print('-----------')

Let now the weak say I have strength.

-----------
By the Spirit of pow'r.

-----------
That raised Christ from the dead.

-----------
Let now the poor stand and confess.
That my portion in Him.

-----------
And I'm more then blessed.

-----------
Let now our hearts burn with a flame.

-----------
A fire consuming all for Your Son's holy name.

-----------
And with the heavens we declare.

-----------
You are our King.

-----------
We love You, Lord.

-----------
We Worship You.

-----------
You are our God.

-----------
You alone are good.

-----------
Let now Your church shine as your bride.

-----------
That You saw in Your heart.

-----------
As You offered up Your life.

-----------
Let now the lost be welcome home.

-----------
By the saved and redeemed.

-----------
Those adopted as Your own.

-----------
Let now our hearts burn with a flame.

-----------
A fire consuming all for Your Son's holy name.

-----------
And with the heavens we declare.

-----------
You are our King.



In [95]:
url = 'https://language.googleapis.com/v1beta2/documents:analyzeSentiment?key='+apikey

data = {"document": {"type":"PLAIN_TEXT","content":lyric_final},"encodingType":"UTF8"}

response = requests.post(url,json=data)
json.loads(response.text)


{'documentSentiment': {'magnitude': 17.1, 'score': 0.3},
 'language': 'en',
 'sentences': [{'sentiment': {'magnitude': 0.3, 'score': -0.3},
   'text': {'beginOffset': 0,
    'content': 'Let now the weak say I have strength.'}},
  {'sentiment': {'magnitude': 0.3, 'score': 0.3},
   'text': {'beginOffset': 38, 'content': "By the Spirit of pow'r."}},
  {'sentiment': {'magnitude': 0.1, 'score': 0.1},
   'text': {'beginOffset': 62,
    'content': 'That raised Christ from the dead.'}},
  {'sentiment': {'magnitude': 0.7, 'score': -0.7},
   'text': {'beginOffset': 96,
    'content': 'Let now the poor stand and confess.'}},
  {'sentiment': {'magnitude': 0, 'score': 0},
   'text': {'beginOffset': 132, 'content': 'That my portion in Him.'}},
  {'sentiment': {'magnitude': 0.8, 'score': 0.8},
   'text': {'beginOffset': 156, 'content': "And I'm more then blessed."}},
  {'sentiment': {'magnitude': 0.2, 'score': 0.2},
   'text': {'beginOffset': 183,
    'content': 'Let now our hearts burn with a flame.

In [96]:
json.loads(response.text)

{'documentSentiment': {'magnitude': 17.1, 'score': 0.3},
 'language': 'en',
 'sentences': [{'sentiment': {'magnitude': 0.3, 'score': -0.3},
   'text': {'beginOffset': 0,
    'content': 'Let now the weak say I have strength.'}},
  {'sentiment': {'magnitude': 0.3, 'score': 0.3},
   'text': {'beginOffset': 38, 'content': "By the Spirit of pow'r."}},
  {'sentiment': {'magnitude': 0.1, 'score': 0.1},
   'text': {'beginOffset': 62,
    'content': 'That raised Christ from the dead.'}},
  {'sentiment': {'magnitude': 0.7, 'score': -0.7},
   'text': {'beginOffset': 96,
    'content': 'Let now the poor stand and confess.'}},
  {'sentiment': {'magnitude': 0, 'score': 0},
   'text': {'beginOffset': 132, 'content': 'That my portion in Him.'}},
  {'sentiment': {'magnitude': 0.8, 'score': 0.8},
   'text': {'beginOffset': 156, 'content': "And I'm more then blessed."}},
  {'sentiment': {'magnitude': 0.2, 'score': 0.2},
   'text': {'beginOffset': 183,
    'content': 'Let now our hearts burn with a flame.

Spotify API
https://developer.spotify.com/web-api/console/get-search-item/

In [329]:
#Get Artist
search = "michael card".replace(" ","%20")
header = {"Authorization":"Bearer "+auth_token}    

url = 'https://api.spotify.com/v1/search?q='+search+'&type=artist'
print(url)

artist = requests.get(url,headers=header).json()
# print(artist)
popularity = 0 
for i in artist['artists']['items']:
    if i['popularity'] > popularity:
        popularity = i['popularity']
        artist = [i['id'],i['name'],i['genres'],i['popularity'],i['followers']['total']]
print(artist)    
#     print(i['genres'])
#     print(i['popularity'])
#     print(i['followers']['total'])
#     print(i['name'])
#     print(i['id'])
#     print('--------------')

https://api.spotify.com/v1/search?q=michael%20card&type=artist
['35ufIa5SJNtsQMIKrQeYfN', 'Michael Card', ['ccm', 'christian alternative rock', 'christian christmas', 'christian music', 'christian relaxative', 'deep christian rock', 'worship'], 43, 10330]


In [299]:
# Get Albums
columns = ['album_name','album_id']
dfalbums = pd.DataFrame(columns=columns)

url = 'https://api.spotify.com/v1/artists/'+artist[0]+"/albums"
print(url)

albums = requests.get(url,headers=header).json()
# If there is no next page, potentially stopped here!

iter = 0
while (albums['next'] != None):
#     print(albums['next'])
    iter = iter + 1
    if iter > 20 : break
    for a in albums['items']:
        row = [a['name'],a['id']]
        dfalbums.loc[len(dfalbums)] = row  
    albums = requests.get(albums['next'],headers=header).json()

print(iter)

https://api.spotify.com/v1/artists/35ufIa5SJNtsQMIKrQeYfN/albums
https://api.spotify.com/v1/artists/35ufIa5SJNtsQMIKrQeYfN/albums?offset=20&limit=20&album_type=single,album,compilation,appears_on,ep
https://api.spotify.com/v1/artists/35ufIa5SJNtsQMIKrQeYfN/albums?offset=40&limit=20&album_type=single,album,compilation,appears_on,ep
https://api.spotify.com/v1/artists/35ufIa5SJNtsQMIKrQeYfN/albums?offset=60&limit=20&album_type=single,album,compilation,appears_on,ep
3


In [301]:
dfalbums.head(3)

,album_name,album_id
0,Best Of Michael Card,7v465J3GY2CoAAfwvSVfCQ
1,Best Of Michael Card,4Ak1i8hx3CKHtolbhBldcI
2,John: A Misunderstood Messiah,7n3Ex6RhoprUdqxvLcEVcy


In [ ]:
# Get Tracks ID in Albums
# Loop all albums

columns = ['album_name','album_id','track_id','track_name','track_artist']
dftracks = pd.DataFrame(columns=columns)

for key,row in dfalbums.iterrows():
#     pause = randint(1,2)
    time.sleep(pause)
#     print(row['album_id'])
    url = 'https://api.spotify.com/v1/albums/'+row['album_id']+"/tracks"
    print(url)
    iter = 0 
    while True:
        tracks = requests.get(url,headers=header).json()
        for t in tracks['items']:
            insert = [row['album_name'],row['album_id'],t['id'],t['name'],t['artists'][0]['name']]
            dftracks.loc[len(dftracks)] = insert
            
        if (tracks['next'] == None)|(iter>5) : 
            break
        else: 
            tracks = requests.get(tracks['next'],headers=header).json()
            iter = iter + 1

    
    

In [331]:
print(len(dftracks[dftracks['track_artist']=='Michael Card']))
print(len(dftracks))

1165
1868


In [ ]:
#TODO
#Loop all Tracks
#Get 50 at a time

list_tracks = []
for t in tracks['items']:
#     print(t['id'])
    list_tracks.append(t['id'])
query_multiple = ','.join(list_tracks)
query_multiple

In [265]:
# Get Tracks Audio Features

# url = '	https://api.spotify.com/v1/audio-analysis/'+"1j0gShwK8aK4pKMAln67ak"
max = 50 
feature_url = '	https://api.spotify.com/v1/audio-features?ids='+query_multiple
track_url = 'https://api.spotify.com/v1/tracks?ids='+query_multiple

track_features = requests.get(feature_url,headers=header).json()
track_details = requests.get(track_url,headers=header).json()


In [ ]:
df = pd.DataFrame(track_features['audio_features']).loc[:,['acousticness','danceability','duration_ms','energy','id',
                                                         'instrumentalness','key','liveness','loudness','mode','speechiness',
                                                         'tempo','time_signature','valence']]
tf = pd.DataFrame(track_details['tracks']).loc[:,['id','name','artists']]

consolidated_features = tf.merge(df,how='inner')
def artist(x):
#     artist_list[artist_list['Genre']=='Band'].iloc[20,0].split('\xa0')[0]
    return x['artists'][0]['name']
    
consolidated_features['singer'] = consolidated_features.apply(lambda x:artist(x),axis=1)
del consolidated_features['artists']
consolidated_features.head(3)


# for tf in track_features['audio_features']:
#     print(tf)
# for td in track_details['tracks']:
#     print(td['id'],td['name'],td['popularity'])

In [ ]:
response.request.headers

Musixmatch
https://developer.musixmatch.com/